In [2]:
import numpy as np
import pandas as pd
from scipy.signal import correlate
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from itertools import product
from itertools import combinations
from datetime import datetime, timedelta
from multiprocessing import Pool


# Load the temporal series data
data_with_date = pd.read_csv('temporal_series_24.csv')
data = data_with_date.drop(columns=['date'])

#Inizialize parameters
max_tau = 15 
threshold = 0.05

#Function to compute maximum of time correlation function between two temporal series
def time_delayed_cross_correlation(df, col1, col2, max_tau, threshold=0.05):
    """
    Optimized custom time-delayed cross-correlation function for two time series.

    Args:
    df (pandas.DataFrame): Dataframe containing the time series data.
    col1 (str): Name of the first time series column.
    col2 (str): Name of the second time series column.
    max_tau (int): Maximum time delay (positive or negative).
    threshold (float): Threshold for maximum correlation.

    Returns:
    float: Maximum cross-correlation above the threshold, or 0 if none.
    """
    # Extract the series and compute means and std deviations
    series_i = df[col1].values
    series_j = df[col2].values
    mean_i, std_i = series_i.mean(), series_i.std()
    mean_j, std_j = series_j.mean(), series_j.std()
    
    # Check if standard deviations are zero to avoid division by zero
    if std_i == 0 or std_j == 0:
        return 0
    
    # Initialize array to store cross-correlations for each lag
    cross_corrs = np.zeros(2 * max_tau + 1)

    # Compute cross-correlation for each lag
    for tau in range(-max_tau, max_tau + 1):
        if tau < 0:
            # Positive lag: shift series_j forward by -tau (series_i aligns with delayed series_j)
            numerator = np.sum((series_i[-tau:] - mean_i) * (series_j[:len(series_j) + tau] - mean_j))
            denominator = (std_i * std_j * (len(series_i) + tau))
        else:
            # Negative or zero lag: shift series_i forward by tau (series_j aligns with delayed series_i)
            numerator = np.sum((series_i[:len(series_i) - tau] - mean_i) * (series_j[tau:] - mean_j))
            denominator = (std_i * std_j * (len(series_i) - tau))

        # Calculate cross-correlation, check for zero denominator
        cross_corrs[tau + max_tau] = numerator / denominator if denominator != 0 else 0

    # Find the maximum correlation and compare with threshold
    max_corr = np.max(cross_corrs)
    max_index = np.argmax(cross_corrs)
    best_tau = max_index - max_tau  # Convert array index back to tau value

    # Apply threshold check and adjust max_corr based on tau direction
    if max_corr >= threshold:
        return max_corr if best_tau >= 0 else -max_corr
    else:
        return 0


#Function to compute the whole adjacency matrix (impossible to compute)
def ccr_matrix(df):
    #build the adjacency matrix
    adjacency_matrix = pd.DataFrame(np.nan, index=df.columns, columns=df.columns)

    #iterate for each pair of columns
    for col1 in df.columns:
        for col2 in df.columns:
            adjacency_matrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)

    return adjacency_matrix

#function to compute just a submatrix of the adjacency matrix (submatricese on the diagonal)
def compute_submatrix(df, columns, filename, max_tau):
    submatrix = pd.DataFrame(index=columns, columns=columns)

    # Loop through column pairs, skipping redundant calculations
    for col1, col2 in product(columns, repeat=2):
        if col1 <= col2:  # Ensures each pair is calculated only once
            correlation = time_delayed_cross_correlation(df, col1, col2, max_tau)
            submatrix.loc[col1, col2] = correlation
            submatrix.loc[col2, col1] = correlation  # Fill symmetric position

    submatrix.to_csv(filename)
    print(f"Submatrix {filename} has been saved.")
    return submatrix

#function to computer a submatrix which is not on the diagonal
def compute_cross_group_matrix(df, group_a, group_b, filename):
    cross_matrix = pd.DataFrame(index=group_a, columns=group_b)
    for col1, col2 in product(group_a, group_b):
        cross_matrix.loc[col1, col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    cross_matrix.to_csv(filename)
    print(f"submatrix {filename} has been saved")
    return cross_matrix

#Now to compute the submatrices we need to divide the 3104 columns into groups
def divide_into_groups(df, group_sizes):
    #Divides the comlumns into specified sizes
    columns = df.columns.tolist()
    groups = []
    start = 0
    for size in group_sizes:
        groups.append(columns[start:start + size])
        start += size
    return groups

#Change of program, now we will try to compute manually the single pieces
group_1 = data.columns[0:1000]
group_2 = data.columns[1000:2000]
group_3 = data.columns[2000:3104]


filename_sub_1 = "24_sub_mat_1.csv"
compute_submatrix(data, group_1, filename_sub_1, max_tau)

filename_sub_2 = "24_sub_mat_2.csv"
compute_submatrix(data, group_2, filename_sub_2, max_tau)

filename_sub_3 = "24_sub_mat_3.csv"
compute_submatrix(data, group_3, filename_sub_3, max_tau)

filename_cross_1 = "24_cross_matrix_1_2.csv"
compute_cross_group_matrix(data, group_1, group_2, filename_cross_1)

filename_cross_2 = "24_cross_matrix_1_3.csv"
compute_cross_group_matrix(data, group_1, group_3, filename_cross_2)

filename_cross_3 = "24_cross_matrix_2_3.csv"
compute_cross_group_matrix(data, group_2, group_3, filename_cross_3)


Submatrix 24_sub_mat_1.csv has been saved.
Submatrix 24_sub_mat_2.csv has been saved.
Submatrix 24_sub_mat_3.csv has been saved.
submatrix 24_cross_matrix_1_2.csv has been saved
submatrix 24_cross_matrix_1_3.csv has been saved
submatrix 24_cross_matrix_2_3.csv has been saved


,38099.0,38101.0,38103.0,38105.0,39001.0,39003.0,39005.0,39007.0,39009.0,39011.0,...,56027.0,56029.0,56031.0,56033.0,56035.0,56037.0,56039.0,56041.0,56043.0,56045.0
21083.0,-0.410976,0.412953,0.437176,0.423872,0.420636,-0.301132,-0.351684,-0.216799,0.326608,0.378642,...,-0.176555,0.577662,-0.724996,-0.57297,0.583171,0.685451,0.455477,-0.509896,0.626527,-0.37297
21085.0,0.39596,-0.445981,0.352829,0.440843,0.390905,-0.23473,-0.294632,-0.127661,0.311804,-0.332497,...,-0.155211,-0.571889,-0.686794,-0.621971,0.510364,-0.727704,-0.365572,-0.528952,0.53096,0.392119
21087.0,-0.348752,-0.298269,0.199243,-0.320641,0.352221,-0.232857,-0.200274,-0.14256,-0.326432,-0.322325,...,0.093717,-0.322084,-0.454877,-0.392348,0.294549,-0.433844,0.330838,-0.34326,0.324424,-0.570472
21089.0,0.408774,-0.450871,0.448223,0.464659,-0.435494,-0.437136,-0.386797,-0.354649,-0.442929,-0.485322,...,-0.281783,0.57382,0.655692,0.576723,0.503983,0.666703,0.508555,0.525492,0.500713,0.338447
21091.0,-0.465346,-0.476311,0.426919,-0.458014,-0.482054,-0.3961,-0.34027,-0.381654,0.46514,0.473929,...,-0.30344,-0.470378,-0.632084,-0.534551,0.56203,-0.60259,0.621307,-0.47678,0.561605,0.335983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38089.0,0.758305,0.927093,0.477769,0.886933,-0.846049,-0.69406,-0.668094,-0.538457,-0.712753,-0.756507,...,-0.358784,-0.620562,-0.555795,-0.598491,-0.520137,-0.622202,-0.608776,-0.625184,0.445133,-0.397662
38091.0,0.300749,0.252123,0.358531,0.247994,0.316961,0.494033,0.462427,0.575979,0.414044,0.452297,...,0.498333,0.376145,-0.300299,0.355925,-0.174233,-0.244051,0.398457,-0.220029,0.214881,-0.16995
38093.0,0.671091,0.73447,0.437979,0.755983,-0.690465,-0.546211,-0.531278,-0.429139,-0.607543,-0.599807,...,-0.271718,-0.506995,-0.712349,-0.600165,-0.593769,-0.706007,-0.435838,-0.585646,0.646938,-0.440419
38095.0,0.548029,0.555728,0.520929,0.590264,-0.563264,-0.736823,-0.630995,-0.784764,-0.591796,-0.642529,...,-0.79601,-0.506649,0.344776,-0.53701,-0.451219,-0.391462,0.571205,0.464744,-0.35223,0.258873
